In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import json
from rca import process_categorical, best_logistic_solver, k_fold_cross_val, make_binary_scoring, make_multiclass_scoring, checker
import pickle

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Loading Data

In [2]:
rca = pd.read_csv('../../data/results/rca.csv').dropna()
meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata.csv', index_col=0)
norms = pd.read_csv('../../data/psychNorms/psychNorms.zip', index_col=0, compression='zip', low_memory=False)

# Adding norm_cat to rca
rca['norm_cat'] = (
    rca['norm'].apply(lambda norm: meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

rca

,embed,embed_type,norm,train_n,test_n,p,r2_mean,r2_sd,mse_mean,mse_sd,check,norm_cat
0,CBOW_GoogleNews,text,Freq_HAL,28012,7003,300,0.522118,0.008398,2.715453,0.072527,pass,frequency
1,CBOW_GoogleNews,text,Freq_KF,19285,4822,300,0.500425,0.009710,0.156666,0.004775,pass,frequency
2,CBOW_GoogleNews,text,Freq_SUBTLEXUS,28636,7159,300,0.537265,0.009814,0.361345,0.007803,pass,frequency
3,CBOW_GoogleNews,text,Freq_SUBTLEXUK,29316,7330,300,0.545643,0.008416,0.446042,0.009075,pass,frequency
4,CBOW_GoogleNews,text,Freq_Blog,31876,7969,300,0.523700,0.008448,0.400166,0.009345,pass,frequency
...,...,...,...,...,...,...,...,...,...,...,...,...
7003,THINGS,behavior,familiarity_vanarsdall,376,95,49,0.091050,0.077868,3216.050320,580.598768,pass,familiarity
7004,THINGS,behavior,imageability_vanarsdall,376,95,49,0.074112,0.090266,1317.411401,292.505564,pass,imageability
7005,THINGS,behavior,familiarity_fear,173,44,49,0.151637,0.158767,0.782185,0.220500,pass,familiarity
7006,THINGS,behavior,aoa_fear,173,44,49,0.015012,0.129597,0.514519,0.043528,pass,age of acquisition


In [3]:
embed_avgs = (
    rca[['embed', 'norm_cat', 'r2_mean']]
    .groupby(['embed', 'norm_cat']).median(numeric_only=True) # median is used to mitigate outliers within norm_cats
    .groupby('embed').mean()
    .rename(columns={'r2_mean': 'r2_avg'})
)
embed_avgs

,r2_avg
embed,
CBOW_GoogleNews,0.451043
EEG_speech,-0.065510
EEG_text,-0.048021
GloVe_CommonCrawl,0.315571
GloVe_Twitter,0.298494
GloVe_Wikipedia,0.301758
LexVec_CommonCrawl,0.293098
PPMI_SVD_EAT,0.296026
PPMI_SVD_SWOW,0.403018


In [4]:
# Adding embed types
with open('../../data/embed_to_dtype.json', 'r') as f:
    embed_to_type = json.load(f)
embed_avgs['type'] = embed_avgs.index.map(embed_to_type)

# Finding top 2 text 
text_name_1, text_name_2 = (
    embed_avgs.query('type == "text"').sort_values('r2_avg', ascending=False).head(2).index.tolist()
)
text_name_1, text_name_2

('CBOW_GoogleNews', 'fastText_CommonCrawl')

In [5]:
# Finding top behavior
behavior_name = (
    embed_avgs.query('type == "behavior"').sort_values('r2_avg', ascending=False).head(1).index[0]
)
behavior_name

'PPMI_SVD_SWOW'

In [6]:
# Loading embeds
text_1 = pd.read_csv(f'../../data/embeds/{text_name_1}.csv', index_col=0)
text_2 = pd.read_csv(f'../../data/embeds/{text_name_2}.csv', index_col=0)
behavior = pd.read_csv(f'../../data/embeds/{behavior_name}.csv', index_col=0)

with open('../../data/brain_behav_union.pkl', 'rb') as f:
    brain_behav_union = pickle.load(f)

# Subsdetting to brain and behavior union and aligning vocabs
intersect = sorted(list(set.intersection(
    set(text_1.index), set(text_2.index), set(behavior.index), set(brain_behav_union)
)))
text_1, text_2, behavior = text_1.loc[intersect], text_2.loc[intersect], behavior.loc[intersect]

# Standardizing
standardize = lambda df: (df - df.mean()) / df.std()
text_1, text_2, behavior = standardize(text_1), standardize(text_2), standardize(behavior)

# Ensembling for comparison
embeds = {
    behavior_name: behavior,
    text_name_1: text_1, 
    text_name_2: text_2,
    text_name_1 + '&' + text_name_2: pd.concat([text_1, text_2], axis=1),
    text_name_1 + '&' + behavior_name: pd.concat([text_1, behavior], axis=1),
    text_name_2 + '&' + behavior_name: pd.concat([text_2, behavior], axis=1)
}

# Fixing column names
for embed_name, embed in embeds.items():
    embed.columns = list(range(embed.shape[1]))
    embeds[embed_name] = embed

{name: embed.shape for name, embed in embeds.items()}

{'PPMI_SVD_SWOW': (11723, 300),
 'CBOW_GoogleNews': (11723, 300),
 'fastText_CommonCrawl': (11723, 300),
 'CBOW_GoogleNews&fastText_CommonCrawl': (11723, 600),
 'CBOW_GoogleNews&PPMI_SVD_SWOW': (11723, 600),
 'fastText_CommonCrawl&PPMI_SVD_SWOW': (11723, 600)}

In [7]:
# Changing associated_embed to more usable format
meta['associated_embed'] = meta['associated_embed'].str.split(' ')

# Log transforming selected norms
norms_to_log = pd.read_csv('../../data/norms_to_log.csv')['norm']
norms[norms_to_log] = norms[norms_to_log].apply(np.log1p)
norms_to_log

0             Nsenses_WordNet
1           Nsenses_Wordsmyth
2         Nmeanings_Wordsmyth
3          Nmeanings_Websters
4                   NFeatures
5                       Sem_N
6         Assoc_Freq_Token123
7                 Cue_SetSize
8           LexicalD_RT_V_ELP
9           LexicalD_RT_V_ECP
10          LexicalD_RT_V_BLP
11         LexicalD_RT_A_MALD
12         LexicalD_RT_A_AELP
13              Naming_RT_ELP
14       SemanticD_RT_Calgary
15                  rt_khanna
16                     rt_ley
17               rt_chiarello
18                    rt_chen
19             aoa_rt_cortese
20    imageability_rt_cortese
21                  rt_schock
Name: norm, dtype: object

## Cross Validation

In [8]:
# Ridge
min_ord, max_ord = -5, 5
alphas = np.logspace(
    min_ord, max_ord, max_ord - min_ord + 1
)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scoring = make_binary_scoring()
multiclass_scoring = make_multiclass_scoring()
continuous_scoring = {'r2': 'r2', 'neg_mse': 'neg_mean_squared_error'}

# outer_cv setting 
outer_cv, n_jobs = 5, 6

solo_embed_names = [text_name_1, text_name_2, behavior_name] # For checking data leakage in checker

In [9]:
# RCA
rca = []
for norm_name in tqdm(norms.columns):
    print(f'{norm_name}:')
    y = norms[norm_name].dropna()
    
    to_print = []
    for embed_name, embed in embeds.items():
        
        # Aligning embed with norm
        X, y = embed.align(y, axis='index', join='inner', copy=True)
        
        # Checking norm dtype 
        norm_dtype = meta.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators ir categorical or continuous
        if norm_dtype in ['binary', 'multiclass']: # categorical
            X, y = process_categorical(outer_cv, inner_cv, X, y)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(y.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(y, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv), solver=solver
            )
            scoring = binary_scoring if norm_dtype == 'binary' else multiclass_scoring
        else: # continuous
            estimator, scoring = ridge, continuous_scoring
            
        # Cross validation
        associated_embed = meta.loc[norm_name, 'associated_embed']
        check = checker(solo_embed_names, y, norm_dtype, associated_embed, outer_cv)
        if check == 'pass':
            scores = k_fold_cross_val(estimator, X, y, outer_cv, scoring, n_jobs) # stratification is automatically used for classification
            r2s, mses = scores['test_r2'], - scores['test_neg_mse']
        else:
            r2s, mses = pd.Series([np.nan] * outer_cv), pd.Series([np.nan] * outer_cv)
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(y))
        for i, (r2, mse) in enumerate(zip(r2s, mses)):
            rca.append([embed_name, norm_name, train_n, i + 1, r2, mse, check])
            
        # Printing
        to_print.append([embed_name, r2s.mean(), r2s.std(), check])
    to_print = pd.DataFrame(to_print, columns=['embed', 'r2_mean', 'r2_std', 'check'])
    print(to_print.sort_values('r2_mean', ascending=False).head(10).reset_index(drop=True))
    print('--------------------------------')
 
 
rca = pd.DataFrame(
    rca, columns=[
        'embed', 'norm', 'train_n', 'fold', 'r2', 'mse', 'check']
)
rca.to_csv('../../data/results/rca_ensemb.csv', index=False)
rca

  0%|          | 0/292 [00:00<?, ?it/s]

Freq_HAL:
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) inst

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

...

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.238483  0.021816  pass
1                         PPMI_SVD_SWOW  0.238322  0.018760  pass
2         CBOW_GoogleNews&PPMI_SVD_SWOW  0.233109  0.021987  pass
3  CBOW_GoogleNews&fastText_CommonCrawl  0.223876  0.012682  pass
4                  fastText_CommonCrawl  0.210081  0.026418  pass
5                       CBOW_GoogleNews  0.168465  0.047997  pass
--------------------------------
Sem_Diversity:
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) inst

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

                                  embed   r2_mean    r2_std check
0  CBOW_GoogleNews&fastText_CommonCrawl  0.576822  0.070480  pass
1                  fastText_CommonCrawl  0.565732  0.137874  pass
2                       CBOW_GoogleNews  0.551297  0.074591  pass
3    fastText_CommonCrawl&PPMI_SVD_SWOW  0.535748  0.220551  pass
4         CBOW_GoogleNews&PPMI_SVD_SWOW  0.472984  0.178464  pass
5                         PPMI_SVD_SWOW  0.462997  0.290269  pass
--------------------------------
imagery_toronto:
                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.669332  0.056579  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.649006  0.039851  pass
2         CBOW_GoogleNews&PPMI_SVD_SWOW  0.640592  0.052086  pass
3                  fastText_CommonCrawl  0.620359  0.057510  pass
4                       CBOW_GoogleNews  0.579855  0.039124  pass
5                         PPMI_SVD_SWOW  0.578119  0.078404  pass
--------------------------

,embed,norm,train_n,fold,r2,mse,check
0,PPMI_SVD_SWOW,Freq_HAL,9040,1,0.478024,1.691392,pass
1,PPMI_SVD_SWOW,Freq_HAL,9040,2,0.507218,1.634118,pass
2,PPMI_SVD_SWOW,Freq_HAL,9040,3,0.549852,1.669096,pass
3,PPMI_SVD_SWOW,Freq_HAL,9040,4,0.500720,1.695633,pass
4,PPMI_SVD_SWOW,Freq_HAL,9040,5,0.540526,1.796841,pass
...,...,...,...,...,...,...,...
8755,fastText_CommonCrawl&PPMI_SVD_SWOW,imageability_fear,510,1,0.667308,0.565868,pass
8756,fastText_CommonCrawl&PPMI_SVD_SWOW,imageability_fear,510,2,0.640832,0.705835,pass
8757,fastText_CommonCrawl&PPMI_SVD_SWOW,imageability_fear,510,3,0.592154,0.793009,pass
8758,fastText_CommonCrawl&PPMI_SVD_SWOW,imageability_fear,510,4,0.539998,0.741900,pass
